In [1]:
import cv2
import numpy as np
import imutils

def transformImage(img):
    img_pre_processed = preProcessImage(img)
    end_result = processImage(img, img_pre_processed)
    return end_result
    
def preProcessImage(img):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_gray_blur = cv2.GaussianBlur(img_gray, (15, 15), 0)
    img_gray_blur_canny = cv2.Canny(img_gray_blur, 20, 100)
    
    pts = np.array([[0,717], [0,560], [610,430], [710,430], [1200, 717]])
    mask = np.zeros(img_gray_blur_canny.shape[0:2], dtype=np.uint8)
    cv2.drawContours(mask, [pts], -1, (255, 255, 255), -1, cv2.LINE_AA)
  
    res = cv2.bitwise_and(img_gray_blur_canny,img_gray_blur_canny,mask = mask)
   
    return res

def processImage(img, pre_processed_img):
    img_lines = np.zeros_like(img)
    linesP = cv2.HoughLinesP(pre_processed_img, 1, np.pi/180, 100, None, 50, 10)
    if linesP is not None:
        for i in range(0, len(linesP)):
            l = linesP[i][0]  
            cv2.line(img_lines, (l[0], l[1]), (l[2], l[3]), (0, 0, 255), 4, cv2.LINE_AA)
    end_result = cv2.addWeighted(img , 0.8, img_lines, 1, 0)
    return end_result

def processVideo(video_cap, video_out):
    k = 0
    while(video_cap.isOpened()):
        ret, frame = video_cap.read()
        if(ret == True):
            k+=1
            new_frame = transformImage(frame)
            new_frame = postProcessImage(new_frame, frame, k)
            output.write(new_frame)
        else:
            cap.release()
            
def postProcessImage(img, old_img, fps):
    smaller_img = imutils.resize(old_img, height = 200)
    x_offset = 925
    y_offset = 520
    img[y_offset:y_offset+smaller_img.shape[0], x_offset:x_offset+smaller_img.shape[1]] = smaller_img
    cv2.putText(img, str(fps), (50,50), cv2.FONT_HERSHEY_DUPLEX, 1, (255, 255, 255), 1)
    return img

cap = cv2.VideoCapture('roadvideo.mp4')
if(cap.isOpened() == False):
    print("Error opening video file ")
    
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
frame_size = (frame_width,frame_height)
fps = 30
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
output = cv2.VideoWriter('output.mp4', fourcc, fps, frame_size)

processVideo(cap, output)

KeyboardInterrupt: 